# Import dependencies

In [ ]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [1]:
import time
import gc

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from semeval_reader import SemevalReader

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from torch import cuda

from models.BERT_Dropout_CNN_BiLSTM_Linear import BERT_Dropout_CNN_BiLSTM_Linear

In [2]:
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [4]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [5]:
def get_target_list_for_polarity(polarity):
    if polarity == 'positive':
        return [0, 0, 1]
    if polarity == 'negative':
        return [1, 0, 0]
    return [0, 1, 0]

In [6]:
semeval_reader = SemevalReader('../../../data/semeval16_restaurants_train.xml')

reviews = semeval_reader.read_reviews()
absolute_polarity_sentences = semeval_reader.get_absolute_polarity_sentences()

df = pd.DataFrame(map(lambda x: (x.text, x.opinions[0].polarity), absolute_polarity_sentences))
df.rename(columns={0: 'text'}, inplace=True)
df['target_list'] = df.apply(lambda row: get_target_list_for_polarity(row[1]), axis=1)

absolute_polarity_df = df.drop(columns=[1])

# Train & Validate

In [7]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [8]:
BERT_FINE_TUNED_PATH = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned.pth'

In [9]:
MODEL_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_cnn_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_cnn_bilstm_linear.csv'

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
best_accuracy = 0.0

In [12]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [13]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [14]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [15]:
for i in range(10):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/10")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True
    )

    model = BERT_Dropout_CNN_BiLSTM_Linear(bert=torch.load(BERT_FINE_TUNED_PATH), bert_seq_len=512, dropout=0.3, bilstm_in_features=256, no_out_labels=3, conv_out_channels=54, conv_kernel_size=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    outputs = np.argmax(outputs, axis=1)
    targets = np.argmax(targets, axis=1)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.bert, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/313, Loss: 0.704782247543335
Epoch: 0/2, Batch: 31/313, Loss: 0.3756673038005829
Epoch: 0/2, Batch: 62/313, Loss: 0.4134715497493744
Epoch: 0/2, Batch: 93/313, Loss: 0.27588051557540894
Epoch: 0/2, Batch: 124/313, Loss: 0.10756982862949371
Epoch: 0/2, Batch: 155/313, Loss: 0.14092975854873657
Epoch: 0/2, Batch: 186/313, Loss: 0.04903667792677879
Epoch: 0/2, Batch: 217/313, Loss: 0.042280733585357666
Epoch: 0/2, Batch: 248/313, Loss: 0.0980050191283226
Epoch: 0/2, Batch: 279/313, Loss: 0.1028352677822113
Epoch: 0/2, Batch: 310/313, Loss: 0.0424124114215374
Epoch: 1/2, Batch: 0/313, Loss: 0.03761627525091171
Epoch: 1/2, Batch: 31/313, Loss: 0.05013922601938248
Epoch: 1/2, Batch: 62/313, Loss: 0.04470596835017204
Epoch: 1/2, Batch: 93/313, Loss: 0.047271355986595154
Epoch: 1/2, Batch: 124/313, Loss: 0.030281921848654747
Epoch: 1/2, Batch: 155/313, Loss: 0.007951546460390091
Epoch: 1/2, Batch: 186/313, Loss: 0.09534861147403717
Epoch: 1/2, Batch: 217/313, Loss

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6912798881530762
Epoch: 0/2, Batch: 31/313, Loss: 0.5216327905654907
Epoch: 0/2, Batch: 62/313, Loss: 0.23990842700004578
Epoch: 0/2, Batch: 93/313, Loss: 0.5302728414535522
Epoch: 0/2, Batch: 124/313, Loss: 0.14416447281837463
Epoch: 0/2, Batch: 155/313, Loss: 0.057487986981868744
Epoch: 0/2, Batch: 186/313, Loss: 0.062488652765750885
Epoch: 0/2, Batch: 217/313, Loss: 0.6953446865081787
Epoch: 0/2, Batch: 248/313, Loss: 0.03751008212566376
Epoch: 0/2, Batch: 279/313, Loss: 0.03658119589090347
Epoch: 0/2, Batch: 310/313, Loss: 0.2706473469734192
Epoch: 1/2, Batch: 0/313, Loss: 0.06847523152828217
Epoch: 1/2, Batch: 31/313, Loss: 0.04361281543970108
Epoch: 1/2, Batch: 62/313, Loss: 0.016039982438087463
Epoch: 1/2, Batch: 93/313, Loss: 0.020024631172418594
Epoch: 1/2, Batch: 124/313, Loss: 0.006968831177800894
Epoch: 1/2, Batch: 155/313, Loss: 0.029733693227171898
Epoch: 1/2, Batch: 186/313, Loss: 0.006245831958949566
Epoch: 1/2, Batch: 217/313,

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 4/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6877774596214294
Epoch: 0/2, Batch: 31/313, Loss: 0.4450756907463074
Epoch: 0/2, Batch: 62/313, Loss: 0.14852657914161682
Epoch: 0/2, Batch: 93/313, Loss: 0.07804758846759796
Epoch: 0/2, Batch: 124/313, Loss: 0.4141598641872406
Epoch: 0/2, Batch: 155/313, Loss: 0.05709226429462433
Epoch: 0/2, Batch: 186/313, Loss: 0.024300839751958847
Epoch: 0/2, Batch: 217/313, Loss: 0.4383622407913208
Epoch: 0/2, Batch: 248/313, Loss: 0.03575683385133743
Epoch: 0/2, Batch: 279/313, Loss: 0.0527932308614254
Epoch: 0/2, Batch: 310/313, Loss: 0.02556406334042549
Epoch: 1/2, Batch: 0/313, Loss: 0.028780248016119003
Epoch: 1/2, Batch: 31/313, Loss: 0.02240573614835739
Epoch: 1/2, Batch: 62/313, Loss: 0.0650378093123436
Epoch: 1/2, Batch: 93/313, Loss: 0.023644499480724335
Epoch: 1/2, Batch: 124/313, Loss: 0.0287962444126606
Epoch: 1/2, Batch: 155/313, Loss: 0.01022975891828537
Epoch: 1/2, Batch: 186/313, Loss: 0.12686768174171448
Epoch: 1/2, Batch: 217/313, Loss:

...

Run 8/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6769095659255981
Epoch: 0/2, Batch: 31/313, Loss: 0.4932368993759155
Epoch: 0/2, Batch: 62/313, Loss: 0.215643048286438
Epoch: 0/2, Batch: 93/313, Loss: 0.06575541198253632
Epoch: 0/2, Batch: 124/313, Loss: 0.2117094248533249
Epoch: 0/2, Batch: 155/313, Loss: 0.04946407675743103
Epoch: 0/2, Batch: 186/313, Loss: 0.47523388266563416
Epoch: 0/2, Batch: 217/313, Loss: 0.41523677110671997
Epoch: 0/2, Batch: 248/313, Loss: 0.40831610560417175
Epoch: 0/2, Batch: 279/313, Loss: 0.06275638937950134
Epoch: 0/2, Batch: 310/313, Loss: 0.7252015471458435
Epoch: 1/2, Batch: 0/313, Loss: 0.020068658515810966
Epoch: 1/2, Batch: 31/313, Loss: 0.4545806050300598
Epoch: 1/2, Batch: 62/313, Loss: 0.022932125255465508
Epoch: 1/2, Batch: 93/313, Loss: 1.015676498413086
Epoch: 1/2, Batch: 124/313, Loss: 0.03146810084581375
Epoch: 1/2, Batch: 155/313, Loss: 0.017712537199258804
Epoch: 1/2, Batch: 186/313, Loss: 0.023177701979875565
Epoch: 1/2, Batch: 217/313, Loss: 

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 9/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6873453855514526
Epoch: 0/2, Batch: 31/313, Loss: 0.481809139251709
Epoch: 0/2, Batch: 62/313, Loss: 0.4168526530265808
Epoch: 0/2, Batch: 93/313, Loss: 0.08546923100948334
Epoch: 0/2, Batch: 124/313, Loss: 0.07506592571735382
Epoch: 0/2, Batch: 155/313, Loss: 0.4628526568412781
Epoch: 0/2, Batch: 186/313, Loss: 0.0583287850022316
Epoch: 0/2, Batch: 217/313, Loss: 0.05127903074026108
Epoch: 0/2, Batch: 248/313, Loss: 0.056613773107528687
Epoch: 0/2, Batch: 279/313, Loss: 0.03440272435545921
Epoch: 0/2, Batch: 310/313, Loss: 0.2415875494480133
Epoch: 1/2, Batch: 0/313, Loss: 0.4254055917263031
Epoch: 1/2, Batch: 31/313, Loss: 0.06977089494466782
Epoch: 1/2, Batch: 62/313, Loss: 0.013398069888353348
Epoch: 1/2, Batch: 93/313, Loss: 0.05119992420077324
Epoch: 1/2, Batch: 124/313, Loss: 0.01102224737405777
Epoch: 1/2, Batch: 155/313, Loss: 0.02521286904811859
Epoch: 1/2, Batch: 186/313, Loss: 0.008346736431121826
Epoch: 1/2, Batch: 217/313, Loss: 

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6797058582305908
Epoch: 0/2, Batch: 31/313, Loss: 0.3630525469779968
Epoch: 0/2, Batch: 62/313, Loss: 0.28373003005981445
Epoch: 0/2, Batch: 93/313, Loss: 0.08483363687992096
Epoch: 0/2, Batch: 124/313, Loss: 0.10023200511932373
Epoch: 0/2, Batch: 155/313, Loss: 0.35300713777542114
Epoch: 0/2, Batch: 186/313, Loss: 0.3283767104148865
Epoch: 0/2, Batch: 217/313, Loss: 0.033448707312345505
Epoch: 0/2, Batch: 248/313, Loss: 0.16090860962867737
Epoch: 0/2, Batch: 279/313, Loss: 0.039309024810791016
Epoch: 0/2, Batch: 310/313, Loss: 0.314700186252594
Epoch: 1/2, Batch: 0/313, Loss: 0.06322813034057617
Epoch: 1/2, Batch: 31/313, Loss: 0.03328830003738403
Epoch: 1/2, Batch: 62/313, Loss: 0.027376320213079453
Epoch: 1/2, Batch: 93/313, Loss: 0.39311879873275757
Epoch: 1/2, Batch: 124/313, Loss: 0.37061607837677
Epoch: 1/2, Batch: 155/313, Loss: 0.024305399507284164
Epoch: 1/2, Batch: 186/313, Loss: 0.021119065582752228
Epoch: 1/2, Batch: 217/313, Los

In [16]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.939103,0.939103,0.956482,0.939103,0.671292,0.939103,0.684426,221.394408
1,0.948718,0.948718,0.622823,0.948718,0.647908,0.948718,0.635071,200.959013
2,0.951923,0.951923,0.630926,0.951923,0.643415,0.951923,0.637095,198.442935
3,0.929487,0.929487,0.617116,0.929487,0.635479,0.929487,0.625972,197.734812
4,0.935897,0.935897,0.606391,0.935897,0.651017,0.935897,0.625725,197.809759
5,0.923077,0.923077,0.607782,0.923077,0.635933,0.923077,0.621538,198.355403
6,0.929487,0.929487,0.599011,0.929487,0.645815,0.929487,0.618994,206.877071
7,0.935897,0.935897,0.612381,0.935897,0.652033,0.935897,0.630326,206.529027
8,0.935897,0.935897,0.610148,0.935897,0.649735,0.935897,0.628555,205.707866
9,0.894231,0.894231,0.674234,0.894231,0.777483,0.894231,0.671732,212.221633


In [17]:
results.to_csv(STATS_OUTPUT)